<a href="https://colab.research.google.com/github/YongWanJin/KDTteam5_semiproject/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%BD%94%EB%A1%9C%EB%82%98%EC%A0%84%ED%9B%84%EB%A7%A4%EC%B6%9C%EC%95%A1%EB%B3%80%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 코드1 : 로컬에서 자료 불러오기
#   (그런데 Colab에서는 어떻게 하는지 모르겠습니다...ㅠㅠ)
#   사용한 데이터 : 서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_2017년~2021년

import os
os.chdir("C:\\Users\\PC\\Desktop\\Statistics\\2022_K-Digital-Traning\\0. Semi Project\\Data\\매출액")


In [ ]:
### 코드2 : 전처리 - 여러개의 csv를 하나의 데이터프레임'total'으로 병합

from glob import glob      # glob : csv들을 하나로 모아주는 모듈
import pandas as pd

file_names = glob("*.csv") # 해당 경로에 있는 모든 csv파일 리스트를 생성

total = pd.DataFrame()     # total : 병합이 이루어질 데이터프레임

for file_name in file_names : # 리스트 내 요소에 하나씩 접근
    temp = pd.read_csv(file_name, encoding='CP949') # 하나씩 불러옴
    total = pd.concat([total, temp])                # total에 하나씩 병합

total.reset_index(inplace=True, drop=True)          # 인덱스 초기화
del(temp)

print(total.info()) # 결과물
print(total)

In [ ]:
### 코드3 : 전처리 - 필요한 행만 추출한 데이터프레임'ts'를 생성

ts = total[["기준_년_코드", "기준_분기_코드", "상권_구분_코드_명",
                    "상권_코드", "상권_코드_명",
                    "서비스_업종_코드","서비스_업종_코드_명", "분기당_매출_금액"]]

In [ ]:
### 코드4 : 전처리 - 서비스 업종 코드의 문자를 없앤 뒤, 숫자로 변환

ts['서비스_업종_코드'] = ts['서비스_업종_코드']\
    .str.replace(pat=r'[CS]',repl=r'', regex=True)\
        .astype('int')

In [ ]:
### 코드5 : 전처리 - 'ts'에서 식당 업종만 추출해서 새로운 데이터프레임'ts_food'에 저장

ts_food = ts[ ts['서비스_업종_코드'] <= 100010]  # 서비스업종코드 100001~100010이 식당업계

print(ts_food.info()) # 결과물
print(ts_food)

In [ ]:
### 코드6 : 각 상권별 매출 평균을 분기별로 입력할 수 있는 데이터프레임 껍데기 'means' 생성
## 열 인덱스 : 연도와 분기
## 행 인덱스 : 상권코드와 상권코드명
## 필드값 : 해당 연도&분기, 해당 상권의 매출 평균

years = [i for i in ts_food.기준_년_코드.unique() for _ in range(4)]                      # 상위 열인덱스 : 연도
quarter = [i for i in ts_food.기준_분기_코드.unique()]*len(ts_food.기준_년_코드.unique()) # 하위 열인덱스 : 분기
res_c = [i for i in ts_food['상권_코드'].unique()]                                        # 상위 행인덱스 : 상권코드
res_c_name = [i for i in ts_food['상권_코드_명'].unique()]                                # 하위 행인덱스 : 상권코드명
means = pd.DataFrame(index = [res_c, res_c_name],
                  columns = [years, quarter])
means.index.names = ["상권_코드", "상권_코드_명"]
means.columns.names = ["기준_년_코드", "기준_분기_코드"]
means.sort_values(by="상권_코드", inplace=True)    # 상권코드 순으로 정렬

In [ ]:
### 코드7 : 매출 평균을 계산 후 'means'에 입력

for y in range(5) :      # 연도 루프
    for q in range(4) :  # 분기 루프
        
        # 2017+y년도 1+q분기 상권별 매출 평균 계산
        means_q = ts_food[(ts_food.기준_년_코드 == 2017+y)&(ts_food.기준_분기_코드 == 1+q)]\
            .groupby('상권_코드_명')['분기당_매출_금액'].mean()
        means_q = pd.DataFrame(means_q)
        
        # 2017+y년도 1+q분기에 존재하는 상권코드명 추출
        names_q = ts_food[(ts_food.기준_년_코드 == 2017+y)&(ts_food.기준_분기_코드 == 1+q)]\
            ['상권_코드_명'].unique()
        names_q = pd.Series(names_q)
        
        # 2017+y년도 1+q분기에 존재하는 상권코드 추출
        c_q = ts_food[(ts_food.기준_년_코드 == 2017+y)&(ts_food.기준_분기_코드 == 1+q)]\
            ['상권_코드'].unique()
        c_q = pd.Series(c_q)
        
        # 상권코드, 상권코드명 병합 후, 매출평균means_q의 순서에 맞게 재배치
        c_q = pd.concat([c_q, names_q], axis=1)
        c_q.rename(columns={0:'상권_코드', 1:'상권_코드_명'}, inplace=True)
        c_q.sort_values(by='상권_코드_명', inplace=True)
        
        # 매출평균과 상권코드&상권코드명 join 후, 상권코드 순으로 오름차순 배치
        means_q = pd.merge(c_q, means_q, right_on='상권_코드_명', left_on='상권_코드_명')
        means_q.rename(columns = {'분기당_매출_금액':'분기당_매출_금액_평균'}, inplace=True)
        means_q.sort_values(by='상권_코드', inplace=True, ignore_index=True)

        # 2017+y년도 1+q분기 매출 평균을 means에 차례대로 입력
        hd_means, hd_means_q = 0, 0
        while hd_means < len(means) :
            if means.index[hd_means][1] == means_q.at[hd_means_q, '상권_코드_명'] :
                means.iat[hd_means, 4*y+q] = means_q.at[hd_means_q, '분기당_매출_금액_평균']
                hd_means += 1
                hd_means_q += 1
            else :
                hd_means += 1

del(hd_means)
del(hd_means_q)
del(c_q)
del(means_q)
del(names_q)
del(y)
del(q)

print(means.info()) # 결과물
print(means)

In [ ]:
### 코드8 : 코로나 이전 대비 코로나 이후 매출이 심하게 많이 떨어진 도시 선정
### 지표 : 2019년 3,4분기 평균매출 대비 2020년 1,2분기 평균매출 증가율
### 해당 지표를 데이터프레임 'rate_inc_table'에 저장

# 전처리 - 결측치 처리

# 2019년 3,4분기와 2020년 1,2분기가 하나라도 없는 것들 제거
# 상권 데이터 1625개 중 29개 삭제됨.
i = 0
while i < len(means) :
    if means.iloc[i,10:14].isnull().sum() :
        means.drop([means.index[i]], inplace=True)
        continue
    else :
        i += 1

# 평균매출 증가율 계산 및 입력
rate_inc_table = pd.DataFrame(index = means.index,
                              columns = ['코로나전후_매출증가율'])
rate_inc_table.index.names = ["상권_코드", "상권_코드_명"]  #  데이터프레임 'rate_inc_table' 생성

for i in range(len(means)) :
    before = (means.iat[i,10]+means.iat[i,11])/2  # 코로나 직전(2019년 3,4분기) 평균
    after = (means.iat[i,12]+means.iat[i,13])/2   # 코로나 직후(2020년 1,2분기) 평균
    rate_inc = (after-before)/before*100          # 코로나 직전대비 코로나 직후 평균매출 증가율 공식
    rate_inc_table.iat[i,0] = rate_inc            # 'rate_inc_table'에 평균매출 증가율 입력   
del(before)
del(after)

# 코로나 직전 대비 코로나 직후 평균매출 증가율이 낮은것부터 정렬
# (심각하게 타격을 입은 상권 순으로 정렬)
rate_inc_table.sort_values(by='코로나전후_매출증가율', inplace=True) 

# 이상치 1건 삭제
# (상권코드명'신림초등학교' 혼자 평균매출 증가율 314%라는 이상한 수치임;;;)
rate_inc_table = rate_inc_table[rate_inc_table['코로나전후_매출증가율']<300]



In [ ]:
### 앞으로 해야할것

# 1. 코로나전후 평균매출 증가율을 지도시각화
# : 심각한 순서대로 진한색깔(눈에띄는색깔)로 표기
# 전체 평균과 심각한 지역을 비교하는 것도 잊지말고.

# 2. 그중에서 심각한 지역을 위주로, 회복이 더딘 지역 선정
# 사용할 지표 : 2020년 1분기 이후부터 평균매출 증가율의 변화 추세(시계열)

# 3, 2번에서 했던걸 선 그래프로 시각화
# : 전체 평균적인 추세랑 심각한 지역의 추세를 동시에 표기하여 비교
